In [ ]:
import numpy as np
import PIL
from collections import OrderedDict
from types import SimpleNamespace

import torch
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import utils.logging as logging
import utils.metrics as metrics
from models.model import GLPDepth
from dataset.base_dataset import get_dataset
from configs.test_options import TestOptions
import matplotlib.pyplot as plt

ckpt_path = "../models/kitti_swin_large.ckpt"


In [ ]:
""" 
glp_args = SimpleNamespace(
    max_depth=80,
    backbone="swin_large",
    depths=[2, 2, 18, 2],
    window_size=[22, 22, 22, 11],
    pretrain_window_size=[12, 12, 12, 6],
    drop_path_rate=0.3,
    use_checkpoint=False,
    use_shift=[True, True, False, False],
    pretrained="",
    num_deconv=3,
    num_filters=[32, 32, 32],
    deconv_kernels=[2, 2, 2],
)
"""
glp_args = SimpleNamespace(
    max_depth=80,
    backbone="swin_large",
    depths=[2, 2, 18, 2],
    window_size=[22, 22, 22, 11],
    pretrain_window_size=[12, 12, 12, 6],
    drop_path_rate=0.3,
    use_checkpoint=False,
    use_shift=[True, True, False, False],
    pretrained="",
    num_deconv=3,
    num_filters=[32, 32, 32],
    deconv_kernels=[2, 2, 2],
)

model = GLPDepth(glp_args).to("cpu")
model_weight = torch.load(ckpt_path)
if 'module' in next(iter(model_weight.items()))[0]:
    model_weight = OrderedDict((k[7:], v) for k, v in model_weight.items())
model.load_state_dict(model_weight)
model.eval()

In [ ]:
print(model.decoder.to("cuda"))

In [ ]:
input_RGB = np.array(PIL.Image.open("../data/tower0.png"))
model_input = transforms.ToTensor()(input_RGB).unsqueeze(0).float()
pred = model(model_input)

In [ ]:
print(pred["pred_d"].shape)
depth = pred["pred_d"].permute(0, 2, 3, 1).squeeze(0)
depth = depth.detach().numpy()
print(depth.shape)
plt.imshow(depth, cmap="gray")

In [ ]:
depth.max()